In [1]:
import compute_convergence
import data
import glob
import seaborn as sns
import numpy as np
sns.set_theme()

# Data for experiments

In [33]:
debugdata = data.load_data("../output/synthetic/s1d/")
debugdata = data.postprocess(debugdata)

In [65]:
debugdata.shape

(3799, 834)

Filter cases where ratio > 0.95 and save to pickle

In [70]:
dta = debugdata[debugdata.th == 1e-8].copy()
print(dta.shape)
dta['idx095'] = dta.apply(lambda row: np.where(row[[f"ratio_{i}" for i in range(200)]]>0.95)[0], axis=1)
dta['pns095'] = dta.apply(lambda row: row[[f"pns_{i}" for i in row.idx095]].values, axis=1)
dta['idx098'] = dta.apply(lambda row: np.where(row[[f"ratio_{i}" for i in range(200)]]>0.98)[0], axis=1)
dta['pns098'] = dta.apply(lambda row: row[[f"pns_{i}" for i in row.idx098]].values, axis=1)

dta = dta[np.abs(dta.pns_l_emcc - dta.pns_u_emcc) > 0.02]
outdata=dta[['pns095', 'pns098', "pns_l_emcc", "pns_u_emcc", "pns_l_ccve", "pns_u_ccve", 'modelFile', ]]

(248, 834)


In [67]:
dta.shape

(199, 838)

In [71]:
dta['pns095'].apply(len).describe()

count    199.000000
mean     189.412060
std       28.700316
min        9.000000
25%      198.500000
50%      200.000000
75%      200.000000
max      200.000000
Name: pns095, dtype: float64

In [72]:

outdata.to_pickle("pns_kl_1e8_s1d.pickle")